In [ ]:
import pickle
import pandas as pd
import hashlib

import numpy as np
import os
from ete3 import NCBITaxa
from datetime import datetime

from dotenv import load_dotenv
import ontology_mapper

import shutil
from tqdm import tqdm

from IPython.display import clear_output


## File paths

In [ ]:


PARENT_DATA_PATH = '../data'
DATA_PATH = '../data/genelab_data_in_csv_format.pickle'
FILENAME = '20231220.csv'



## Setting Bioportal API and checking the directories

In [ ]:
load_dotenv('../.env')
apikey = os.getenv("BIOPORTAL_API_KEY")
if not apikey:
    raise Exception("BIOPORTAL_API_KEY is not set in the .env file!")

NEO4J_DATA = os.getenv('NEO4J_DATA')
node_dir = os.path.join(NEO4J_DATA, 'nodes')
rel_dir = os.path.join(NEO4J_DATA, 'relationships')

shutil.rmtree(node_dir, ignore_errors=True)
shutil.rmtree(rel_dir, ignore_errors=True)

os.makedirs(NEO4J_DATA, exist_ok=True)
os.makedirs(node_dir, exist_ok=True)
os.makedirs(rel_dir, exist_ok=True)




## Get Metadata based on the selection criteria

In [ ]:
spoke_relevant_assays = ['RNA Sequencing (RNA-Seq)', 'DNA microarray', 'mass spectrometry', 'nucleotide sequencing', '16S']
organism_to_exclude = ['Viridiplantae', 'Fungi']

metadata_df = pd.read_csv(os.path.join(PARENT_DATA_PATH, FILENAME), header=1)

ncbi = NCBITaxa()
lineage_superkingdom = [None]*metadata_df.shape[0]
lineage_kingdom = [None]*metadata_df.shape[0]
for index, row in metadata_df.iterrows():
    try:
        tax_id = row['organism.term accession number'].split('/')[-1]
        lineage = ncbi.get_lineage(tax_id)
        lineage_names = ncbi.get_taxid_translator(lineage)
        for item in lineage_names:
            rank = list(ncbi.get_rank([item]).values())[0]
            if rank == 'superkingdom':
                lineage_superkingdom[index] = lineage_names[item]
            if rank == 'kingdom':
                lineage_kingdom[index] = lineage_names[item]
    except:
        continue
metadata_df.loc[:, 'lineage_superkingdom'] = lineage_superkingdom
metadata_df.loc[:, 'lineage_kingdom'] = lineage_kingdom

metadata_df = metadata_df[metadata_df['GL-processed']!='False']
metadata_df_ = metadata_df[metadata_df['study assay technology type'].isin(spoke_relevant_assays)]
metadata_df_without_plants_fungi = metadata_df_[~metadata_df_['lineage_kingdom'].isin(organism_to_exclude)]
metadata_df_without_plants_fungi = metadata_df_without_plants_fungi[metadata_df_without_plants_fungi.organism != 'Not Applicable']
metadata_df_without_plants_fungi.dropna(subset=['lineage_kingdom'], inplace=True)
metadata_df_without_plants_fungi.dropna(subset=['organism'], inplace=True)

metadata_df_without_plants_fungi_ = metadata_df_without_plants_fungi.dropna(subset=['material type.term accession number'])
metadata_df_without_plants_fungi_.loc[:,'material_type_id'] = metadata_df_without_plants_fungi_['material type.term accession number'].apply(lambda x:x.split('/')[-1].split('_')[0])

metadata_df_without_plants_fungi_after_uberon_filter = metadata_df_without_plants_fungi_[(metadata_df_without_plants_fungi_.material_type_id.str.startswith('D')) | 
                                   (metadata_df_without_plants_fungi_.material_type_id == 'UBERON') |
                                 (metadata_df_without_plants_fungi_['material_type_id'].str.contains('fma'))|
                                 (metadata_df_without_plants_fungi_['material_type_id'].str.contains('EHDA')) |
                                 (metadata_df_without_plants_fungi_['material_type_id'].str.contains('owl'))]

clear_output()


## Open saved Genelab csv files

In [ ]:
%%time

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

In [ ]:
study_ids = list(map(lambda x:x['metadata']['accession'], data))
assay_ids = list(map(lambda x:'GLDS'+x.split('OSD')[-1], study_ids))


In [ ]:
assay_factor_list = []
for index,item in enumerate(data):
    factors_list = []
    for col in list(item['data'].columns):
        if 'Log2fc_' in col:
            factors_list.append(col.split('Log2fc_')[-1])
    assay_factor_list.append((assay_ids[index], factors_list))
            
assay_factor_list_df = pd.DataFrame(assay_factor_list, columns=['assay_id', 'factor_list'])
assay_factor_list_df
                    

In [ ]:


assay_unique_factor_list = []

for index, row in assay_factor_list_df.iterrows():
    item_list = row.factor_list
    unique_list = []
    reverse_list = []
    for item in item_list:
        if item in reverse_list:
            continue
        else:
            unique_list.append(item)
            item_split = item.split(')v(')
            item_split[0] = item_split[0]+')'
            item_split[1] = '('+item_split[1]
            reverse_list.append('v'.join(reversed(item_split)))
    unique_list_new = []
    for item in unique_list:
        if ('(Space Flight' in item) & (item.startswith('(Ground Control')):
            item_split = item.split(')v(')
            new_item = '(' + item_split[1] + 'v' + item_split[0] + ')'
            unique_list_new.append(new_item)
        else:
            unique_list_new.append(item)
    assay_unique_factor_list.append((row['assay_id'], unique_list_new))
    
assay_unique_factor_list_df = pd.DataFrame(assay_unique_factor_list, columns=['assay_id', 'factor_list'])
assay_unique_factor_list_df.loc[:,'accession'] = study_ids

assay_unique_factor_list_df = assay_unique_factor_list_df.explode('factor_list')

assay_unique_factor_list_df = pd.merge(assay_unique_factor_list_df, metadata_df_without_plants_fungi_after_uberon_filter[['accession', 'assay name', 'study assay technology type']], on='accession').drop_duplicates(subset=['assay_id', 'factor_list']).reset_index().drop('index', axis=1)

assay_unique_factor_list_df.loc[:,'factors_1'] = assay_unique_factor_list_df.factor_list.apply(lambda x:x.split(')v(')[0][1:].split('&'))
assay_unique_factor_list_df.loc[:,'factors_2'] = assay_unique_factor_list_df.factor_list.apply(lambda x:x.split(')v(')[1][0:-1].split('&'))


## Considering Assay nodes with only significant measurements


In [ ]:
assay_unique_factor_list_with_significant_p_value = []
for index, row in tqdm(assay_unique_factor_list_df.iterrows()):
    data_index = np.where(row['accession']==np.array(study_ids))[0][0]
    df = data[data_index]['data']
    df = df[df['Adj.p.value_'+row['factor_list']] < 0.05]
    if df.shape[0] > 0:
        assay_unique_factor_list_with_significant_p_value.append(row)

assay_unique_factor_list_df = pd.DataFrame(assay_unique_factor_list_with_significant_p_value)


6it [00:00, 45.85it/s]

11it [00:00, 34.28it/s]

21it [00:00, 56.53it/s]

30it [00:00, 65.32it/s]

38it [00:00, 66.73it/s]

...

2508it [06:23,  7.26it/s]

2509it [06:24,  6.49it/s]

2510it [06:24,  6.02it/s]

2515it [06:24, 13.66it/s]

2529it [06:24,  6.58it/s]

In [ ]:
def strip_strings_in_list(lst):
    return [s.strip() for s in lst]

assay_unique_factor_list_df['factors_1'] = assay_unique_factor_list_df['factors_1'].apply(strip_strings_in_list)
assay_unique_factor_list_df['factors_2'] = assay_unique_factor_list_df['factors_2'].apply(strip_strings_in_list)


In [ ]:
grouped = metadata_df_without_plants_fungi_after_uberon_filter.groupby('accession')[['material type']].agg(list)

new_rows = []

for index, rows in grouped.iterrows():
    materials = rows['material type']
    if len(materials) == 1:
        new_rows.append([rows.name, materials[0], None])
    elif len(materials) == 2:
        new_rows.append([rows.name, materials[0], materials[1]])
        
    

metadata_df_with_material_type = pd.DataFrame(new_rows, columns=['accession', 'material_1', 'material_2'])

clear_output()


In [ ]:
assay_node_df = pd.merge(assay_unique_factor_list_df, metadata_df_with_material_type, on='accession')

assay_node_df.loc[:,'factor_list_hash'] = assay_node_df.factor_list.apply(lambda x:hashlib.md5(x.encode()).hexdigest())

assay_node_df.loc[:,'identifier'] = assay_node_df.assay_id + '-' + assay_node_df.factor_list_hash

assay_node_df.rename(columns={'assay name':'name', 
                              'study assay technology type':'technology'
                             }, inplace=True)



In [ ]:
study_df = assay_node_df[['accession']]
study_df.rename(columns={'accession':'identifier'}, inplace=True)
study_df.loc[:,'name'] = ''
study_df.loc[:,'organism'] = ''
study_df.loc[:,'taxonomy'] = ''
study_df.loc[:,'strain'] = ''
study_df.loc[:,'duration'] = ''
study_df.loc[:,'duration_unit'] = ''
study_df.drop_duplicates(subset=['identifier'], inplace=True)

SpA_df = assay_node_df[['accession', 'identifier']].drop_duplicates()
SpA_df.rename(columns={'accession':'from', 'identifier':'to'}, inplace=True)

clear_output()

In [ ]:
assay_node_df.drop(['assay_id', 'factor_list_hash', 'factor_list', 'accession'], axis=1, inplace=True)
assay_node_df.loc[:,'measurement'] = ''
assay_node_df = assay_node_df[['identifier', 'name', 'technology', 'measurement', 'factors_1', 'factors_2', 'material_1', 'material_2']]

assay_node_df['material_2'].fillna(assay_node_df['material_1'], inplace=True)

clear_output()

In [ ]:
%%time
assay_node_df = ontology_mapper.map_ontology(assay_node_df, "material_1", "material_id_1", "UBERON", apikey)
assay_node_df = ontology_mapper.map_ontology(assay_node_df, "material_2", "material_id_2", "UBERON", apikey)


## Creating edge files

In [ ]:
AiA_df_1 = assay_node_df[['identifier', 'material_id_1']]
AiA_df_1.rename(columns={'identifier':'from', 'material_id_1':'to'}, inplace=True)
AiA_df_2 = assay_node_df[['identifier', 'material_id_2']]
AiA_df_2.rename(columns={'identifier':'from', 'material_id_2':'to'}, inplace=True)
AiA_df = pd.concat([AiA_df_1, AiA_df_2], ignore_index=True).drop_duplicates()

clear_output()


## Anatomy and Celltype nodes

In [ ]:
material_df = AiA_df[['to']].drop_duplicates()
material_df.rename(columns={'to':'identifier'}, inplace=True)
anatomy_df = material_df[material_df.identifier.str.startswith('UBERON')]
celltype_df = material_df[material_df.identifier.str.startswith('CL')]

AiCT_df = AiA_df[AiA_df.to.str.startswith('CL')]
AiA_df = AiA_df[AiA_df.to.str.startswith('UBERON')]




In [ ]:
assay_node_df.identifier = assay_node_df.identifier.astype('str')
anatomy_df.identifier = anatomy_df.identifier.astype('str')
celltype_df.identifier = celltype_df.identifier.astype('str')
study_df.identifier = study_df.identifier.astype('str')
AiA_df['from'] = AiA_df['from'].astype('str')
AiA_df['to'] = AiA_df['to'].astype('str')
AiCT_df['from'] = AiCT_df['from'].astype('str')
AiCT_df['to'] = AiCT_df['to'].astype('str')
SpA_df['from'] = SpA_df['from'].astype('str')
SpA_df['to'] = SpA_df['to'].astype('str')

assay_node_df.drop_duplicates(subset=['identifier'], inplace=True)
anatomy_df.drop_duplicates(subset=['identifier'], inplace=True)
celltype_df.drop_duplicates(subset=['identifier'], inplace=True)
study_df.drop_duplicates(subset=['identifier'], inplace=True)

AiA_df.drop_duplicates(inplace=True)
AiCT_df.drop_duplicates(inplace=True)
SpA_df.drop_duplicates(inplace=True)

clear_output()


## Saving node and edge files

In [ ]:
today_date = datetime.today()
formatted_date = today_date.strftime("%Y-%d-%m")


assay_node_df.to_csv(os.path.join(node_dir, f'Assay_{formatted_date}.csv'), index=False)
anatomy_df.to_csv(os.path.join(node_dir, f'Anatomy_{formatted_date}.csv'), index=False)
celltype_df.to_csv(os.path.join(node_dir, f'CellType_{formatted_date}.csv'), index=False)
study_df.to_csv(os.path.join(node_dir, f'Study_{formatted_date}.csv'), index=False)

AiA_df.to_csv(os.path.join(rel_dir, f'Assay-INVESTIGATED_AiA-Anatomy_{formatted_date}.csv'), index=False)
AiCT_df.to_csv(os.path.join(rel_dir, f'Assay-INVESTIGATED_AiCT-CellType_{formatted_date}.csv'), index=False)
SpA_df.to_csv(os.path.join(rel_dir, f'Study-PERFORMED_SpA-Assay_{formatted_date}.csv'), index=False)
